# Analisis Inferencial
¿Existen diferencias significativas en la preferencia por formatos de lectura (físico o digital) entre quienes visitaron bibliotecas o librerías con sus padres en su infancia y quienes no?

In [1]:
# - Cargando los datos
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

M19 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2019.csv')
M20 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2020.csv', encoding='latin1')
M21 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2021.CSV', encoding='latin1')
M22 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2022.CSV', encoding='latin1')
M23 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2023.CSV', encoding='latin1')
M24 = pd.read_csv('/content/drive/MyDrive/DBMA1001B/Datos_molec_2024.csv', encoding='latin1')

dataframes = {'2019': M19, '2020': M20, '2021': M21, '2022': M22, '2023': M23, '2024': M24}

# Limpieza de datos
for anio, df in dataframes.items():
    df = df.drop_duplicates()
    df = df[(df['p8_1'] != 0) & (df['p34_1'] != 0)]
    dataframes[anio] = df
# Nivel de significancia proporcionado
alpha = 0.04

Mounted at /content/drive


- Los datos corresponden a proporciones (o tasas de éxito) entre dos grupos independientes.
- Las muestras son lo suficientemente grandes para aplicar el Teorema del Límite Central (TLC), lo que permite asumir que la distribución muestral de las proporciones es aproximadamente normal (lo cual es necesario para realizar la prueba Z).
- El enfoque de intervalos de confianza es útil para cuantificar el rango de valores donde la verdadera diferencia entre proporciones puede estar.

In [2]:
'''
    Intervalo de confianza para la diferencia de proporciones:
    Proporcion (fisico,biblioteca) vs (digital, no biblioteca)
'''
from statsmodels.stats.proportion import confint_proportions_2indep
from IPython.display import display
dif = []

for anio, df in dataframes.items():
    group_fisico_con_padres = df[(df['p8_1'] == 2) & (df['p34_1'] == 1)]
    n_fisico_con_padres = len(df[df['p34_1'] == 1])
    proporcion_fisico_con_padres = len(group_fisico_con_padres) / n_fisico_con_padres


    group_digital_sin_padres = df[(df['p8_1'] == 1) & (df['p34_1'] == 2)]
    n_digital_sin_padres = len(df[df['p34_1'] == 2])
    proporcion_digital_sin_padres = len(group_digital_sin_padres) / n_digital_sin_padres

    # Diferencia de proporciones
    if n_fisico_con_padres > 0 and n_digital_sin_padres > 0:

        ci_diff_low, ci_diff_high = confint_proportions_2indep(
            len(group_fisico_con_padres), n_fisico_con_padres,
            len(group_digital_sin_padres), n_digital_sin_padres,
            alpha=alpha
        )

        dif.append({
            'Año': anio,
            'Proporción (Físico con Padres)': proporcion_fisico_con_padres,
            'Proporción (Digital sin Padres)': proporcion_digital_sin_padres,
            'IC Diferencia Proporciones (Low)': ci_diff_low,
            'IC Diferencia Proporciones (High)': ci_diff_high
        })
display(pd.DataFrame(dif))

,Año,Proporción (Físico con Padres),Proporción (Digital sin Padres),IC Diferencia Proporciones (Low),IC Diferencia Proporciones (High)
0,2019,0.773381,0.117083,0.592270,0.710188
1,2020,0.819242,0.123377,0.637767,0.743676
2,2021,0.671271,0.200000,0.403651,0.532143
3,2022,0.712209,0.228330,0.416715,0.544033
4,2023,0.677524,0.232218,0.374663,0.509305
5,2024,0.702381,0.205628,0.429246,0.557046


In [ ]:
from scipy import stats
import statsmodels.api as sm
from scipy.stats import norm

''' ------------- Prueba de hipótesis sobre diferencia de proporciones -------------
Primero probar el teorema del límite central, para probar si la distribución de la diferencia de proporciones sigue una normal.

Hipótesis nula (H₀): p₁≤p₂
La proporción de estudiantes que usan formato físico en el Grupo 1 (visitó bibliotecas) es menor o igual que la proporción en el Grupo 2 (no visitó bibliotecas).

Hipótesis alternativa (H₁):p₁>p₂
La proporción de estudiantes que usan formato físico en el Grupo 1 es mayor que en el Grupo 2.
'''

# se crea una funcion para graficar Zcrit y Z

def graficaHipotesis1(z_value):
  x = np.linspace(-4, 4, 1000)
  y = norm.pdf(x)

  z_crit = norm.ppf(1 - 0.04)

  plt.plot(x, y, label='Distribución Normal Estándar', color='black')

  x_re_der = np.linspace(z_crit, 4, 1000)

  plt.fill_between(x_re_der, norm.pdf(x_re_der), color='pink', alpha=0.4)

  plt.axvline(z_crit, color='pink', linestyle='--',label=f'Z crítico = {z_crit:.2f}')
  plt.axvline(z_value, color='blue', linestyle='-', label=f'Z obtenido = {z_value:.2f}')

  plt.title('Distribución Normal Estándar y Área de Rechazo')
  plt.xlabel('Valor Z')
  plt.ylabel('Densidad de Probabilidad')

  plt.legend()
  return plt.show()

for anio, df in dataframes.items():
    grupo1 = df[df['p34_1'] == 1]  # Visitó bibliotecas con padres
    grupo2 = df[df['p34_1'] == 2]  # No visitó bibliotecas con padres

    # Proporción de uso de formato físico en ambos grupos
    p1 = sum(grupo1['p8_1'] == 1) / len(grupo1)
    p2 = sum(grupo2['p8_1'] == 1) / len(grupo2)

    n1 = len(grupo1)
    n2 = len(grupo2)

    tlc = (n1 * p1 >= 5 and n1 * (1 - p1) >= 5 and n2 * p2 >= 5 and n2 * (1 - p2) >= 5)
    if tlc:
        print(f'--- Año {anio} ---')
        print("Se cumple el Teorema del Límite Central, se procede a realizar la prueba de hipótesis.\n")

        successes = [sum(grupo1['p8_1'] == 1), sum(grupo2['p8_1'] == 1)]
        tamanios = [n1, n2]

        z, p_value = sm.stats.proportions_ztest(successes, tamanios, alternative='larger')

        print(f'Estadístico Z: {z}')
        print(f'P-valor: {p_value}')

        if p_value < alpha:
            print("Se rechaza la hipótesis nula: La proporción de uso de formato físico es mayor en el Grupo 1.\n")

        else:
            print("No se rechaza la hipótesis nula: No hay evidencia suficiente para afirmar que el Grupo 1 tiene una proporción mayor.\n")
        graficaHipotesis1(z)
    else:
        print(f'--- Año {anio} ---')
        print(f'El Teorema del Límite Central no se cumple para alguna de las proporciones (n1*p1 = {n1 * p1}, n1*(1-p1) = {n1 * (1 - p1)}, n2*p2 = {n2 * p2}, n2*(1-p2) = {n2 * (1 - p2)}), no se puede realizar la prueba de hipótesis.\n')


Identificación de variables más influyentes en la preferencia por el formato de lectura: selección automática de variables al forzar a que algunos coeficientes se reduzcan a cero, eliminando así variables irrelevantes.

In [ ]:
''' ------------- Técnica de selección de variables ------------
Lasso Regression para identificar las variables más importantes que afectan la preferencia por el formato físico o digital.
'''
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

for anio, df in dataframes.items():
    print(f"--- Procesando el año: {anio} ---")

    X = df[['p34_4', 'p34_2', 'p34_1']] # variables predictoras
    y = df['p8_1'] # variable objetivo

    X = pd.get_dummies(X, drop_first=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    lasso = Lasso(alpha=0.01)
    lasso.fit(X_train_scaled, y_train)

    y_pred_lasso = lasso.predict(X_test_scaled)
    mse_lasso = mean_squared_error(y_test, y_pred_lasso)
    print(f"Error cuadrático medio (Lasso): {mse_lasso}")

    lasso_coefficients = pd.DataFrame({'Variable': X.columns, 'Coeficiente': lasso.coef_})
    print(lasso_coefficients)

    plt.figure(figsize=(8, 6))
    plt.bar(lasso_coefficients['Variable'], lasso_coefficients['Coeficiente'])
    plt.title(f'Importancia de las Variables con Lasso - Año {anio}')
    plt.xlabel('Variable')
    plt.ylabel('Coeficiente')
    plt.xticks(rotation=45)
    plt.show()

